In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/MyDrive/Colab\ Notebooks/Covid-detection/COUGH-SOUND-COV-DETECTION

/content/drive/MyDrive/Colab Notebooks/Covid-detection/COUGH-SOUND-COV-DETECTION


In [11]:
import os

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Colab Notebooks/Kaggle"

In [ ]:
!kaggle datasets download -d praveengovi/coronahack-respiratory-sound-dataset

100% 6.81G/6.82G [01:45<00:00, 43.2MB/s]
100% 6.82G/6.82G [01:45<00:00, 69.2MB/s]


In [ ]:
!unzip coronahack-respiratory-sound-dataset.zip

In [4]:
# First create two folders COVID and HEALTHY
#Add covid cough sound samples from the 'CoronaHack-respiratory-sound-dataset' to COVID FOLDER

import csv
healthy_count=0
covid_pos_count=0
with open('Corona-Hack-Respiratory-Sound-Metadata.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        COVID_STATUS=row[3]
        if(COVID_STATUS=='healthy'):
            healthy_count+=1
        elif(COVID_STATUS=='positive_mild' or COVID_STATUS=='positive_asymp' or COVID_STATUS=='positive_moderate'):
            covid_pos_count+=1
    
print("Healthy => ",healthy_count," Covid => ",covid_pos_count)

Healthy =>  1163  Covid =>  56


In [8]:
with open('Corona-Hack-Respiratory-Sound-Metadata.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
      print(row[3])
      print(row[27])
      print(row[30])
      print(row[31])
      break

COVID_STATUS
DATES
cough-heavy
cough-shallow


In [14]:
import shutil
counter=1
sub_folder=' '
with open('Corona-Hack-Respiratory-Sound-Metadata.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        COVID_STATUS=row[3]

        if(row[27]!=sub_folder):
            sub_folder=row[27]
            counter=1
        
        if(COVID_STATUS=='positive_mild' or COVID_STATUS=='positive_asymp' or COVID_STATUS=='positive_moderate'):
            shutil.copy('CoronaHack-Respiratory-Sound-Dataset'+row[30],'COVID/'+str(sub_folder)+'-h-'+str(counter)+'.wav') 
            shutil.copy('CoronaHack-Respiratory-Sound-Dataset'+row[31],'COVID/'+str(sub_folder)+'-s-'+str(counter)+'.wav')
            counter+=1

In [15]:
len([file for file in os.listdir('COVID')])

112

In [ ]:
# Add the COUGHVID covid samples as well as new samples from coswara to the COVID folder (folders prepared in prepare_dataset_1.ipynb)

In [20]:
for file in os.listdir('COSWARA-NEW-COVID-SAMPLES'):
    shutil.move('COSWARA-NEW-COVID-SAMPLES/'+file,'COVID/'+file)

In [22]:
for file in os.listdir('COUGHVID-COVID-SAMPLES'):
    shutil.move('COUGHVID-COVID-SAMPLES/'+file,'COVID/'+file)

In [24]:
len([file for file in os.listdir('COVID')])

634

In [25]:
#1268 healthy cough sounds are taken from COUGHVID dataset
len([file for file in os.listdir('HEALTHY')])

1268

SPLITTING TO TRAIN, TEST, AND VALIDATION FOLDERS (80%, 10%, 10%)

In [33]:
num=0
for file in os.listdir('COVID'):
    num+=1
    if num<=63:
        shutil.copy('COVID/'+file,'val/COVID/'+file)
    elif num<= 126:
        shutil.copy('COVID/'+file,'test/COVID/'+file)
    else:
        shutil.copy('COVID/'+file,'train/COVID/'+file)

In [34]:
print(" train/COVID =>",len([file for file in os.listdir('train/COVID')]))
print(" test/COVID =>",len([file for file in os.listdir('test/COVID')]))
print(" val/COVID =>",len([file for file in os.listdir('val/COVID')]))

 train/COVID => 508
 test/COVID => 63
 val/COVID => 63


In [35]:
num=0
for file in os.listdir('HEALTHY'):
    num+=1
    if num<=126:
        shutil.copy('HEALTHY/'+file,'val/HEALTHY/'+file)
    elif num<=252:
        shutil.copy('HEALTHY/'+file,'test/HEALTHY/'+file)
    else:
        shutil.copy('HEALTHY/'+file,'train/HEALTHY/'+file)

In [36]:
print(" train/HEALTHY =>",len([file for file in os.listdir('train/HEALTHY')]))
print(" test/HEALTHY =>",len([file for file in os.listdir('test/HEALTHY')]))
print(" val/HEALTHY =>",len([file for file in os.listdir('val/HEALTHY')]))

 train/HEALTHY => 1016
 test/HEALTHY => 126
 val/HEALTHY => 126


### *DATA AUGMENTATATION* - As we have only 634 samples of covid cough sounds data augmentation is done

In [ ]:
!pip install audiomentations

In [41]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import soundfile as sf
import librosa

In [ ]:
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
])

sets = ['train/COVID','test/COVID','val/COVID']

for s in sets:
    for audio_file in os.listdir(s):
        samples, sr = librosa.load(s+'/'+audio_file)
        augmented_sample = augment(samples=samples, sample_rate=sr)
        sf.write(s+'/aug_'+audio_file, augmented_sample, sr)

In [44]:
print(" train/COVID =>",len([file for file in os.listdir('train/COVID')]))
print(" test/COVID =>",len([file for file in os.listdir('test/COVID')]))
print(" val/COVID =>",len([file for file in os.listdir('val/COVID')]))

 train/COVID => 1016
 test/COVID => 126
 val/COVID => 126
